In [1]:
!pip3 install pandas
!pip3 install glob
!pip3 install fuzzywuzzy
!pip3 install python-Levenshtein

ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob


In [2]:
import pandas as pd
from glob import glob
from fuzzywuzzy import process

In [3]:
files = glob('data/*')

In [66]:
products_df = pd.read_csv("other_files/products_list.csv")
products = products_df["product"].to_numpy()
products_df = products_df.rename(columns={"product":"item"})
branches_df = pd.read_excel("other_files/branch_list.xlsx")
branches_df["branch_name"] = branches_df["branch_name"].str.lower()
branches_df= branches_df.set_index(["branch_name"])
branches_df = branches_df.reset_index()
branches_df = branches_df.rename(columns={"branch_name":"branch"})
expenses_df = pd.read_excel("other_files/branch_expenses.xlsx")
expenses_df["total_expenses"] = expenses_df.iloc[:,-4:-1].sum(axis=1)
expenses_df = expenses_df.rename(columns={"branch_name":"branch"})
expenses_df["branch"] = branches_df["branch"].str.lower()
expenses_df = expenses_df.groupby(["branch"]).sum()["total_expenses"].reset_index()

,branch,total_expenses
0,armagh outlet,6109.0
1,ballymoney store,6716.0
2,bargoed outlet,7892.0
3,barry outlet,9377.0
4,bassetlaw outlet,9467.0
...,...,...
81,west somerset branch,16655.0
82,worcestershire branch,11429.0
83,wrexham store,23970.0
84,wyre branch,5842.0


In [48]:
def to_df(location):
    if location[-1] == "v":
        return pd.read_csv(location, low_memory=False)
    else:
        return pd.read_json(location)

def add_location(locations, data_frames):
    for i in range(len(locations)):
        location = files[i][files[i].find("_") + 1 : files[i].find(".")]
        data_frames[i]["Branch"] = location.replace("_", " ").lower()
    return data_frames

def replace_headers(headers):
    new_headers=[]
    for header in headers:
        if header in ["quantity","total_quantity", "quantity_purchased", "total_quantity_purchased"]:
           new_headers.append("quantity")
        elif header in ["item", "product", "sku"]:
            new_headers.append("item")
        else:
            new_headers.append(header)
    return new_headers

def normalize_headers(dataframe):
    current_headers = [header.strip().lower() for header in dataframe.columns.values]
    new_headers = replace_headers(current_headers)
    dataframe.columns = new_headers
    return dataframe

def sort_dataframes(dataframes):
    for dataframe in dataframes:
        dataframe = dataframe.sort_values(by=["year", "month", "day", "hour"])
    return dataframes   

def force_numeric(dataframes):
    for dataframe in dataframes:
        dataframe["year"] = pd.to_numeric(dataframe["year"], errors="coerce")
        dataframe["month"] = pd.to_numeric(dataframe["month"], errors="coerce")
        dataframe["day"] = pd.to_numeric(dataframe["day"], errors="coerce")
        dataframe["quantity"] = pd.to_numeric(dataframe["quantity"], errors="coerce")
        dataframe["amount_in_gbp"] = pd.to_numeric(dataframe["amount_in_gbp"], errors="coerce")
        dataframe = dataframe.dropna(subset=["year", "month", "day", "hour", "quantity", "amount_in_gbp"])
    return dataframes
        
def normalize_strings(dataframes):
    for dataframe in dataframes:
        values = dataframe.item.unique()
        for value in values:
            if value not in products:
                new_value = process.extractOne(value, products)[0]
                dataframe.loc[dataframe.item == value, ["item"]] = new_value
    return dataframes

def get_sales_by_item(dataframes):
    final_df = dataframes[0].groupby(["item", "branch"]).sum()
    for i in range(1,len(dataframes)):
        final_df = pd.concat([final_df, dataframes[i].groupby(["item","branch"]).sum()])
    return final_df

def get_df_from_branch_name(branch): 
    for dataframe in final_dataframes:
        if dataframe["branch"].values[0] == branch:
            return dataframe
        
               
    
    

In [6]:
all_dataframes = [to_df(file) for file in files]
all_dataframes = add_location(files, all_dataframes)
replacement_data = to_df("replacement_data/2013_Bassetlaw_outlet.csv")
replacement_data = add_location(["replacement_data/2013_Bassetlaw_outlet.csv"], [replacement_data])[0]
normalized_replacement_data = normalize_headers(replacement_data)
normalized_dataframes = [normalize_headers(dataframe) for dataframe in all_dataframes]

In [7]:
normalized_dataframes[0] = normalized_dataframes[0].loc[(normalized_dataframes[0])["year"] != 2013]
normalized_dataframes[0] = pd.concat([normalized_dataframes[0], normalized_replacement_data], ignore_index=True)

In [8]:
sorted_normalized_dataframes = sort_dataframes(normalized_dataframes)
sorted_normalized_item_dataframes = normalize_strings(sorted_normalized_dataframes)
final_dataframes = force_numeric(sorted_normalized_item_dataframes)

In [9]:
sales_by_item_df = get_sales_by_item(final_dataframes)[["quantity","amount_in_gbp"]]
sales_by_item_df_reset = sales_by_item_df.reset_index()

In [10]:
sales_by_item_df_with_region_and_county = sales_by_item_df_reset.merge(branches_df, on="branch")
sales_by_item_df_with_region_and_county_and_products = sales_by_item_df_with_region_and_county.merge(products_df,on="item")

In [39]:

most_purchased_product_overall = sales_by_item_df_with_region_and_county_and_products.groupby(["item"]).sum()["quantity"].sort_values(ascending=False)
most_purchased_product_per_region = sales_by_item_df_with_region_and_county_and_products.groupby(["region", "item"]).sum()["quantity"].sort_values(ascending=False).reset_index()
most_purchased_product_per_county =  sales_by_item_df_with_region_and_county_and_products.groupby(["county", "item"]).sum()["quantity"].sort_values(ascending=False).reset_index()
most_purchased_category_overall = sales_by_item_df_with_region_and_county_and_products.groupby(["category"]).sum()["quantity"].sort_values(ascending=False)
most_purchased_category_per_region = sales_by_item_df_with_region_and_county_and_products.groupby(["region", "category"]).sum()["quantity"].sort_values(ascending=False).reset_index()
most_purchased_category_per_county =  sales_by_item_df_with_region_and_county_and_products.groupby(["county", "category"]).sum()["quantity"].sort_values(ascending=False).reset_index()


most_purchased_product_overall.to_csv("cleaned_data/requirement_one/most_purchased_product_overall.csv")
most_purchased_product_per_region.to_csv("cleaned_data/requirement_one/most_purchased_product_per_region.csv")
most_purchased_product_per_county.to_csv("cleaned_data/requirement_one/most_purchased_product_per_county.csv")
most_purchased_category_overall.to_csv("cleaned_data/requirement_one/most_purchased_category_overall.csv")
most_purchased_category_per_region.to_csv("cleaned_data/requirement_one/most_purchased_category_per_region.csv")
most_purchased_category_per_county.to_csv("cleaned_data/requirement_one/most_purchased_category_per_county.csv")

In [40]:
branch_quantity_sold_per_region = sales_by_item_df_with_region_and_county_and_products.groupby(["region", "branch"]).sum()["quantity"].sort_values(ascending=False).reset_index()
branch_quantity_sold_per_county = sales_by_item_df_with_region_and_county_and_products.groupby(["county", "branch"]).sum()["quantity"].sort_values(ascending=False).reset_index()
branch_amount_in_gbp_per_region = sales_by_item_df_with_region_and_county_and_products.groupby(["region", "branch"]).sum()["amount_in_gbp"].sort_values(ascending=False).reset_index()
branch_amount_in_gbp_per_county = sales_by_item_df_with_region_and_county_and_products.groupby(["county", "branch"]).sum()["amount_in_gbp"].sort_values(ascending=False).reset_index()

branch_quantity_sold_per_region.to_csv("cleaned_data/requirement_two/branch_quantity_sold_per_region.csv")
branch_quantity_sold_per_county.to_csv("cleaned_data/requirement_two/branch_quantity_sold_per_county.csv")
branch_amount_in_gbp_per_region.to_csv("cleaned_data/requirement_two/branch_amount_in_gbp_per_region.csv")
branch_amount_in_gbp_per_county.to_csv("cleaned_data/requirement_two/branch_amount_in_gbp_per_county.csv")

In [59]:
top_ten_sales_dataframes = []
for branch in branch_quantity_sold_per_region.head(10)["branch"].values:
    top_ten_sales_dataframes.append(get_df_from_branch_name(branch))
    
top_ten_sales_dataframes_per_hour = []
for dataframe in top_ten_sales_dataframes:
    dataframe = dataframe.groupby(["year","month","day","hour", "branch"]).sum()["quantity"]
    top_ten_sales_dataframes_per_hour.append(dataframe)

top_ten_sales_dataframes_per_hour = pd.concat(top_ten_sales_dataframes_per_hour)

top_ten_sales_dataframes_per_hour.to_csv("cleaned_data/requirement_three/top_ten_sales_dataframes_per_hour.csv")

In [72]:
branch_gross_income = sales_by_item_df_with_region_and_county_and_products.groupby(["branch"]).sum()["amount_in_gbp"].sort_values(ascending=False).reset_index()
branch_gross_income_with_expenses = branch_gross_income.merge(expenses_df, on="branch")
branch_gross_income_with_expenses["profit"] = branch_gross_income_with_expenses["amount_in_gbp"] - branch_gross_income_with_expenses["total_expenses"]

,branch,amount_in_gbp,total_expenses,profit
0,east lindsey branch,3.858013e+10,28424.0,3.858011e+10
1,falkirk branch,3.761827e+10,21569.0,3.761825e+10
2,east cambridgeshire outlet,2.782691e+10,15662.0,2.782690e+10
3,warwickshire branch,2.760695e+10,21808.0,2.760693e+10
4,greater london branch,2.750143e+10,10534.0,2.750142e+10
...,...,...,...,...
80,bedfordshire store,1.142447e+10,10810.0,1.142446e+10
81,rugby branch,1.140690e+10,9725.0,1.140689e+10
82,east dunbartonshire branch,1.139136e+10,5859.0,1.139135e+10
83,armagh outlet,1.135290e+10,6109.0,1.135289e+10


In [78]:
branch_profit = branch_gross_income_with_expenses.drop(columns=["amount_in_gbp","total_expenses"])
branch_profit.to_csv("cleaned_data/requirement_four/branch_profit.csv")